# Select sub-set of optimal predictions
Step of the codes :
- Import modules 
- Define settings 
- Define functions 
- Download pc p1, pc pred and pc obs
- Compute RMSE rec for all pc pred 
- Select members with RMSE rec lower than pc p1 
- build xarray with index in the LHS
- save this xarray as netCDF

# Import Module

In [16]:
# Computational modules 
%matplotlib inline
import xarray as xr
import glob
import os
import numpy as np
import netCDF4
from netCDF4 import Dataset
import pandas as pd
import re
from array import array
from pylab import *
#import geopandas
from eofs.xarray import Eof
from eofs.multivariate.standard import MultivariateEof
import random

# Plotting modules 
import matplotlib.pyplot as plt
#from mpl_toolkits.basemap import Basemap
import pandas.plotting
import matplotlib.ticker as ticker
import seaborn as sns
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.colors import BoundaryNorm
from cartopy.util import add_cyclic_point

# Scikit-learn
from sklearn import linear_model
from sklearn.linear_model import LassoCV, LassoLarsCV, LassoLarsIC
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
from sklearn import preprocessing
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.neural_network import MLPRegressor
from scipy.optimize import minimize
from scipy.optimize import dual_annealing
from sklearn.decomposition import PCA

# Settings

### Variables

In [17]:
variables = ['tas', 'pr', 'psl', 'SW', 'LW']
var_ceres = ['rsdt','rsut', 'rlut']
truncations = [18, 18, 8, 28, 22]
TITLE = 'Multi-variate'
ylabel = '$E_{tot}$'

### Paths

In [18]:
path_official='/data/scratch/globc/peatier/CMIP6/CNRM-CM6-1/CFMIP/amip/'
path_PPE='/data/scratch/globc/peatier/PPE/CNRM-CM6-1_PPE/'
path_files='/data/home/globc/peatier/PPE/CNRMppe_error_decomposition/files/'
path_file_npy = '/data/home/globc/peatier/PPE/CNRMppe_save/PPE/ENSEMBLE2/files/npy/'

# Functions

In [19]:
def MSE(mod, obs, W_rmse_2D) :
    diff_tmp = (mod - obs)**2 * W_rmse_2D
    diff = (diff_tmp.sum(['lat', 'lon']))
    return diff

In [20]:
def reconstruct_X(eofs_combined, pc, nb_dims) :
    X_rec_tmp = np.dot(eofs_combined.transpose(),pc)
    if nb_dims == 3 :
        X_rec = xr.DataArray(X_rec_tmp, 
                        dims=["lon", "lat", "time"]).transpose('time', 'lat', 'lon')
    if nb_dims == 2 :
        X_rec = xr.DataArray(X_rec_tmp, 
                        dims=["lon", "lat"]).transpose('lat', 'lon')
    
    X_rec['lat'] = eofs_combined['lat']
    X_rec['lon'] = eofs_combined['lon']
    return X_rec

In [21]:
def MSE_rec(rec_anom_mod_w, rec_anom_obs_w, Mean, W_rmse_2D) :
    mod = rec_anom_mod_w/W_eof_2D + Mean
    obs = rec_anom_obs_w/W_eof_2D + Mean
    diff = MSE(mod, obs, W_rmse_2D)
    return diff

In [22]:
def get_3D_tas_xarr(path, filename, variables):
#    “”"
#    This function read the netCDF file of monthly data, compute the radiative budget, perform a yearly mean and 
#    return a dataframe
#    “”"
    # First step : download the data into dataframe
    file = xr.open_mfdataset(path+filename,combine='by_coords')
    #
    # Second step : compute the annual average 
    df = file[variables].mean('time', keep_attrs=True)
    tas = df['tas']
    #
    return tas

# Download PCs

In [30]:
path_files = '/data/home/globc/peatier/PPE/CNRMppe_error_decomposition/files/'
path = path_files+'nc/'

pc_PPE = {}
pc_obs = {}
pc_pred = {}
pc_p1 = {}
for var in variables :
    ## PPE
    filename = 'pc_PPE_'+var+'.nc'
    pc_PPE_tmp = xr.open_mfdataset(path+filename,combine='by_coords')
    dims_dict = {'time' : 'members', 'mode' : 'modes'}
    pc_PPE[var] = pc_PPE_tmp.rename_dims(dims_dict)
    
    ## observations
    filename = 'pc_obs_'+var+'.nc'
    pc_obs[var] = xr.open_mfdataset(path+filename,combine='by_coords')
    
    ## p1 - the first line of pc_PPE
    pc_p1[var] = pc_PPE[var]['pcs'][0,:]
    
    ## predictions
    filename = 'pc_pred_'+var+'.nc'
    tmp = xr.open_mfdataset(path+filename,combine='by_coords')
    pc_pred[var] = tmp.rename({'__xarray_dataarray_variable__' : 'pcs'})

# Download EOF solvers

In [31]:
import pickle
path = path_files+'pkl/'

solver = {}
for var in variables :
    # open a file, where you stored the pickled data
    file = open(path+'solver_'+var+'.pkl', 'rb')

    # dump information to that file
    solver[var] = pickle.load(file)

    # close the file
    file.close()

In [32]:
eofs = {}
variances = {}
for var in variables :
    eofs[var] = solver[var].eofsAsCovariance(pcscaling=1)
    variances[var] = solver[var].varianceFraction() 

In [33]:
# Reference simulation
path = path_PPE+'ENSEMBLE1/CNRM-CM6-1_amip_PPE/CNRM-CM6-1_amip_r1i1p1f2/'
filename = 'tas_*_CNRM-CM6-1_amip_*.nc'
p1_amip = get_3D_tas_xarr(path, filename, ['tas'])

In [34]:
lat = p1_amip['lat']
lon = p1_amip['lon']
eofs_nb = arange(1,104,1)
#eofs_xr = {}
eofs_combined = {}

for var in variables :
    eofs_xr = xr.DataArray(eofs[var], 
                   coords={'eofs': eofs_nb,'lat': lat,'lon': lon}, 
                   dims=["eofs", "lat", "lon"])#.to_dataset(name=var)
    ## --Combine the modes for reconstruction
    eofs_combined[var] = eofs_xr

# Compute individual MSEs rec. of pc_pred

In [35]:
path = path_files+'nc/'
Mean={}
for var in variables :
    filename = 'CNRMppe_decomposition_mean_'+str(var)+'.nc'
    Mean_tmp =  xr.open_dataset(path+filename)
    Mean[str(var)] = Mean_tmp[var]

In [36]:
for var in variables :
    W_eof_2D = np.load(path_files+'npy/W_eof_2D_'+str(var)+'.npy')
    W_eof_3D = np.load(path_files+'npy/W_eof_3D_'+str(var)+'.npy')
    W_rmse_2D = np.load(path_files+'npy/W_rmse_2D_'+str(var)+'.npy')

In [37]:
MSE_rec_pred = {}
cpt_trunc = 0
for var in variables :
    print(var)
    MSE_rec_pred[var] = {}
    trunc = truncations[cpt_trunc]
    rec_anom_obs_w = reconstruct_X(eofs_combined[var][0:trunc,:,:], pc_obs[var]['pseudo_pcs'][0:trunc], nb_dims=2)
    cpt=0
    for i in range(0,100000, 1) :
        #print(cpt)
        rec_anom_mod_w = reconstruct_X(eofs_combined[var][0:trunc,:,:], pc_pred[var]['pcs'][0:trunc,i], nb_dims=2)
        tmp = MSE_rec(rec_anom_mod_w, rec_anom_obs_w, Mean[str(var)], W_rmse_2D)
        MSE_rec_pred[var]['LHS_index = '+str(cpt)] = float(tmp)
        cpt+=1
    
    cpt_trunc+=1

tas
pr
psl
SW
LW


In [38]:
pd_MSE_rec_pred = pd.DataFrame(MSE_rec_pred)
pd_MSE_rec_pred

,tas,pr,psl,SW,LW
LHS_index = 0,1.036283,2.074441,7303.059608,62.502192,54.028931
LHS_index = 1,1.100781,1.794938,17824.453216,201.536625,75.285426
LHS_index = 2,1.389311,2.315505,23894.938796,232.236054,79.638473
LHS_index = 3,2.314204,2.818023,58342.423654,164.060385,155.535451
LHS_index = 4,0.807454,2.226161,18417.173351,147.469853,60.276254
...,...,...,...,...,...
LHS_index = 99995,1.127336,1.672224,15515.528772,260.296495,83.500650
LHS_index = 99996,2.075135,2.656810,32881.810908,328.218688,81.674798
LHS_index = 99997,1.370659,1.967545,41896.617199,220.191803,73.506187
LHS_index = 99998,2.490433,1.456155,54928.315895,273.687611,52.929727


# Compute RMSE rec. of pc_p1 

In [39]:
rec_anom_obs_w = {}
cpt_trunc = 0
for var in variables :
    print(var)
    MSE_rec_pred[var] = {}
    trunc = truncations[cpt_trunc]
    rec_anom_obs_w[var] = reconstruct_X(eofs_combined[var][0:trunc,:,:], pc_obs[var]['pseudo_pcs'][0:trunc], nb_dims=2)
    cpt_trunc += 1

tas
pr
psl
SW
LW


In [40]:
MSE_rec_p1 = {}
cpt_trunc = 0
for var in variables :
    trunc = truncations[cpt_trunc]
    rec_anom_mod_w = reconstruct_X(eofs_combined[var][0:trunc,:,:], pc_p1[var][0:trunc], nb_dims=2)
    MSE_rec_p1[var] = MSE_rec(rec_anom_mod_w, rec_anom_obs_w[var], Mean[var], W_rmse_2D)
    cpt_trunc +=1

# Compute multi-variate metric

In [41]:
MSE_rec_pred = pd_MSE_rec_pred.reset_index().drop('index', axis= 1)
MSE_rec_pred

,tas,pr,psl,SW,LW
0,1.036283,2.074441,7303.059608,62.502192,54.028931
1,1.100781,1.794938,17824.453216,201.536625,75.285426
2,1.389311,2.315505,23894.938796,232.236054,79.638473
3,2.314204,2.818023,58342.423654,164.060385,155.535451
4,0.807454,2.226161,18417.173351,147.469853,60.276254
...,...,...,...,...,...
99995,1.127336,1.672224,15515.528772,260.296495,83.500650
99996,2.075135,2.656810,32881.810908,328.218688,81.674798
99997,1.370659,1.967545,41896.617199,220.191803,73.506187
99998,2.490433,1.456155,54928.315895,273.687611,52.929727


In [42]:
## Normaliser par p1 référence
for var in variables :
    print(var)
    MSE_rec_pred[var+'_norm'] = MSE_rec_pred[var]/float(MSE_rec_p1[var])

tas
pr
psl
SW
LW


In [43]:
Etot = []
for i in range(0,100000,1) :
    tmp = MSE_rec_pred.iloc[i]
    tmp_sum = tmp['tas_norm']+tmp['pr_norm']+tmp['psl_norm']+tmp['SW_norm']+tmp['LW_norm']
    tmp_mean = tmp_sum/5
    Etot.append(tmp_mean)

In [44]:
MSE_rec_pred['MSE multi'] = Etot
MSE_rec_pred

,tas,pr,psl,SW,LW,tas_norm,pr_norm,psl_norm,SW_norm,LW_norm,MSE multi
0,1.036283,2.074441,7303.059608,62.502192,54.028931,0.984469,0.208632,0.874837,0.565847,1.338268,0.794410
1,1.100781,1.794938,17824.453216,201.536625,75.285426,1.045742,0.180521,2.135199,1.824557,1.864781,1.410160
2,1.389311,2.315505,23894.938796,232.236054,79.638473,1.319846,0.232876,2.862385,2.102486,1.972603,1.698039
3,2.314204,2.818023,58342.423654,164.060385,155.535451,2.198493,0.283416,6.988864,1.485276,3.852532,2.961716
4,0.807454,2.226161,18417.173351,147.469853,60.276254,0.767081,0.223891,2.206201,1.335078,1.493011,1.205052
...,...,...,...,...,...,...,...,...,...,...,...
99995,1.127336,1.672224,15515.528772,260.296495,83.500650,1.070969,0.168180,1.858612,2.356523,2.068267,1.504510
99996,2.075135,2.656810,32881.810908,328.218688,81.674798,1.971378,0.267202,3.938926,2.971438,2.023042,2.234397
99997,1.370659,1.967545,41896.617199,220.191803,73.506187,1.302126,0.197881,5.018814,1.993446,1.820710,2.066595
99998,2.490433,1.456155,54928.315895,273.687611,52.929727,2.365911,0.146449,6.579887,2.477756,1.311042,2.576209


# Select sub-set of RMSE mutli <= 1

In [45]:
optim = {}
index_list = {}
cpt_trunc = 0
for var in variables :
    optim[var] = []
    index_list[var] = []
    trunc = truncations[cpt_trunc]
    for i in range(0,100000, 1) :
        pred = MSE_rec_pred['MSE multi'][i]
        if pred < 1 :
            optim[var].append(pc_pred[var]['pcs'][0:trunc,i].values)
            index_list[var].append(i)
    cpt_trunc += 1

In [46]:
df_optim = {}
xr_optim = {}
cpt_trunc = 0
for var in variables :
    EOF_list = []
    trunc = truncations[cpt_trunc]
    for i in range(1,(trunc+1), 1) :
        EOF_list.append('EOF '+str(i))
    df_optim[var] = pd.DataFrame(optim[var], columns=EOF_list)
    df_optim[var]['LHS index'] = index_list[var]
    xr_optim[var] = df_optim[var].to_xarray()
    cpt_trunc+=1

In [47]:
xr_optim['SW']

<xarray.Dataset>
Dimensions:    (index: 4989)
Coordinates:
  * index      (index) int64 0 1 2 3 4 5 6 ... 4983 4984 4985 4986 4987 4988
Data variables: (12/29)
    EOF 1      (index) float64 0.4695 0.6992 0.09062 ... 0.5781 0.9984 0.8832
    EOF 2      (index) float64 0.8775 0.9684 1.327 ... 0.6099 -0.078 -0.2368
    EOF 3      (index) float64 0.1919 0.2875 0.8699 ... 0.4801 -0.966 0.09336
    EOF 4      (index) float64 0.9668 1.262 0.1563 ... -0.3862 0.6288 0.4031
    EOF 5      (index) float64 -0.06375 2.648 0.4766 ... 0.9097 -0.21 -0.0326
    EOF 6      (index) float64 -0.2439 -1.046 -0.2373 ... -0.0991 -0.4895 0.1011
    ...         ...
    EOF 24     (index) float64 -0.9858 -0.6529 0.0004231 ... -0.3724 -0.2812
    EOF 25     (index) float64 -0.7312 -0.3137 0.4688 ... -1.315 -0.6266 0.1368
    EOF 26     (index) float64 -0.05202 0.4876 -0.6167 ... 0.3845 0.5098 -0.8066
    EOF 27     (index) float64 0.06454 -0.02026 1.051 ... -0.3634 -0.5352 0.6587
    EOF 28     (index) float64 0.6245 -0.09516 0.2288 ... 0.3054 -0.3046 0.4112
    LHS index  (index) int64 0 14 24 36 41 47 ... 99929 99933 99962 99976 99988

# Save data

In [48]:
path_files = '/data/home/globc/peatier/PPE/CNRMppe_error_decomposition/files/'
path = path_files+'nc/'
for var in variables :
    filename = 'optim_pc_PPE_'+var+'_multi.nc'
    xr_optim[var].to_netcdf(path+filename)